In [6]:
import pandas as pd

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 25)

# Load dataset
df1 = pd.read_csv("train.csv")
df2 = pd.read_csv("test.csv")

In [7]:
# Enforce datatypes and fix UNIX time
def process_dataset(df):
    # Format dates
    df["transDate"] = pd.to_datetime(df["transDate"])
    df["dateOfBirth"] = pd.to_datetime(df["dateOfBirth"], format="%Y-%m-%d")
    df["unixTime"] = pd.to_datetime(df["unixTime"], unit="s")

    # Format categories
    df["business"] = df["business"].astype("category")
    df["category"] = df["category"].astype("category")
    df["gender"] = df["gender"].astype("category")
    df["city"] = df["city"].astype("category")
    df["state"] = df["state"].astype("category")
    df["job"] = df["job"].astype("category")

    # Format strings
    df["creditCardNum"] = df["creditCardNum"].astype(str)
    df["firstName"] = df["firstName"].astype(str)
    df["lastName"] = df["lastName"].astype(str)
    df["street"] = df["street"].astype(str)
    df["zip"] = df["zip"].astype(str)
    df["transNum"] = df["transNum"].astype(str)

    # Format bool
    df["isFraud"] = df["isFraud"].astype(bool)

    # Fix UNIX time
    df["transDate"] = df["unixTime"] + pd.DateOffset(years=7)
    df.drop(columns=["unixTime"], inplace=True)


process_dataset(df1)
process_dataset(df2)

In [8]:
# Separate redundant cardholder data to a separate table

cardholder_cols = [
    "creditCardNum",
    "firstName",
    "lastName",
    "gender",
    "street",
    "city",
    "state",
    "zip",
    "latitude",
    "longitude",
    "cityPop",
    "job",
    "dateOfBirth",
]

cardholders = df1[cardholder_cols].drop_duplicates().reset_index(drop=True)
cardholders.sort_values(by="creditCardNum", inplace=True)
cardholders.set_index("creditCardNum", inplace=True)
cardholders["lastTransaction"] = ""
cardholders["meanPurchasePrice"] = 0.0
cardholders["numberOfTransactions"] = 0
cardholders["sumSquareDiffs"] = 0.0
cardholders["historyOfFraud"] = False

cardholders.to_csv("cardholders.csv", index=False)

to_drop = [
    "firstName",
    "lastName",
    "gender",
    "street",
    "city",
    "state",
    "zip",
    "latitude",
    "longitude",
    "cityPop",
    "job",
    "dateOfBirth",
]

transactions = df1.copy()
transactions.set_index("transNum", inplace=True)
transactions.drop(columns=to_drop, inplace=True)

In [9]:
import csv
import os

from vincenty import vincenty_inverse


def distance_from_home(trans_row, cardholder_row):
    coords_home = (cardholder_row["longitude"], cardholder_row["latitude"])
    coords_purchase = (trans_row["merchLongitude"], trans_row["merchLatitude"])

    return vincenty_inverse(coords_home, coords_purchase).km


def distance_from_last(last, current):
    coords_last = (last["merchLongitude"], last["merchLatitude"])
    coords_current = (current["merchLongitude"], current["merchLatitude"])

    return vincenty_inverse(coords_last, coords_current).km


def time_since_last(last, current):
    return (current["transDate"] - last["transDate"]).seconds


file_path = "new_col_data.csv"

if os.path.exists(file_path):
    os.remove(file_path)
file = open(file_path, "a")
handler = csv.writer(file)
handler.writerow(
    ["transNum", "distanceFromHome", "distanceFromLast", "secondsSinceLast", "priceSD"]
)


for index, transaction in transactions.iterrows():
    credit_card_num = transaction["creditCardNum"]
    if credit_card_num in cardholders.index:
        cardholder = cardholders.loc[credit_card_num]
    else:
        raise Exception("Cardholder not found")

    if cardholder["numberOfTransactions"] == 0:
        cardholder["meanPurchasePrice"] = transaction["amount"]
        cardholder["numberOfTransactions"] = 1
        cardholder["sumSquareDiffs"] = 0.0
    else:
        cardholder["meanPurchasePrice"] = (
            cardholder["meanPurchasePrice"] * cardholder["numberOfTransactions"]
            + transaction["amount"]
        ) / (cardholder["numberOfTransactions"] + 1)
        cardholder["numberOfTransactions"] += 1
        cardholder["sumSquareDiffs"] += (
            transaction["amount"] - cardholder["meanPurchasePrice"]
        ) ** 2

    priceSD = (cardholder["sumSquareDiffs"] / cardholder["numberOfTransactions"]) ** 0.5

    distanceFromHome = distance_from_home(transaction, cardholder)

    if cardholder["lastTransaction"] != "":
        lastTransaction = transactions.loc[cardholder["lastTransaction"]]
        distanceFromLast = distance_from_last(lastTransaction, transaction)
        secondsSinceLast = time_since_last(lastTransaction, transaction)
    else:
        distanceFromLast = -1
        secondsSinceLast = -1

    handler.writerow(
        [index, distanceFromHome, distanceFromLast, secondsSinceLast, priceSD]
    )

    cardholder["lastTransaction"] = index
    if not transaction["isFraud"]:
        cardholders.loc[credit_card_num] = cardholder
    else:
        cardholder = cardholders.loc[credit_card_num]
        cardholder["historyOfFraud"] = True
        cardholders.loc[credit_card_num] = cardholder

file.close()

/var/folders/dy/nrpbxz756vv3_l8mp02k87k80000gn/T/ipykernel_47700/3914039861.py:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/dy/nrpbxz756vv3_l8mp02k87k80000gn/T/ipykernel_47700/3914039861.py:45: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/dy/nrpbxz756vv3_l8mp02k87k80000gn/T/ipykernel_47700/3914039861.py:46: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/dy/nrpbxz756vv3_l8mp02k87k80000gn/T/ipy

In [11]:
print(transactions)
print(cardholders)

                                           transDate        creditCardNum  \
transNum                                                                    
3fc1791ff1db94ce9540262d7eb973e4 2019-01-01 04:28:00     4119762878330980   
edbf65b8606b53fc48168c2daa5933e5 2019-01-01 04:28:15       36078114201167   
2fcbde131cff833751dde53da54da6da 2019-01-01 04:29:14     4449530933957320   
bcae564f9a1a299dfbd1efd917499968 2019-01-01 04:29:18     4302475216404890   
5e42e35c255edb55a192b3f1f0600936 2019-01-01 04:32:09      377234009633447   
...                                              ...                  ...   
16cfc30968a4ffc36f44a8fa4d98224f 2019-12-15 13:48:38  4958589671582720000   
9c68a7b61b6d9a1afd3db343f6943e0e 2019-12-15 13:48:53     2576709887791550   
e715a05b1aab3b771d6c22a0ef0e6fcd 2019-12-15 13:49:05     6011679934075340   
df05203f1050a2621bd8c4e41a3e4d8b 2019-12-15 13:49:20  4102003771126570000   
e54fed36ad03af47c786c3b56fd1c326 2019-12-15 13:49:26     4629451965224800   

In [ ]:
import plotly.express as px

fig = px.scatter_geo(
    transactions[transactions["isFraud"] == True],
    lat="merchLatitude",
    lon="merchLongitude",
    color="isFraud",
    scope="usa",
)
fig.update_layout(title="Fraudulent Transaction Map", title_x=0.5)
fig.show(renderer="iframe")

In [25]:
grouped_transactions = transactions.groupby("category")

total_transactions = grouped_transactions.size()
fraud_transactions = grouped_transactions["isFraud"].sum()

fraud_rate = (fraud_transactions / total_transactions) * 100
fraud_rate

/var/folders/dy/nrpbxz756vv3_l8mp02k87k80000gn/T/ipykernel_47700/1548752224.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



category
entertainment     0.325758
food_dining       0.154943
gas_transport     0.578761
grocery_net       0.350653
grocery_pos       1.767254
health_fitness    0.192791
home              0.190268
kids_pets         0.206237
misc_net          1.806350
misc_pos          0.418932
personal_care     0.298039
shopping_net      2.310110
shopping_pos      1.047730
travel            0.300725
dtype: float64

In [37]:
grouped_transactions = df1.groupby("business")

total_transactions = grouped_transactions.size()
fraud_transactions = grouped_transactions["isFraud"].sum()

fraud_rate = (fraud_transactions / total_transactions) * 100
fraud_rate

/var/folders/dy/nrpbxz756vv3_l8mp02k87k80000gn/T/ipykernel_47700/1679477126.py:1: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



business
fraud_Abbott-Rogahn                 0.411523
fraud_Abbott-Steuber                0.000000
fraud_Abernathy and Sons            0.000000
fraud_Abshire PLC                   0.787402
fraud_Adams, Kovacek and Kuhlman    0.806452
                                      ...   
fraud_Zemlak Group                  0.520833
fraud_Zemlak, Tillman and Cremin    0.000000
fraud_Ziemann-Waters                0.458716
fraud_Zieme, Bode and Dooley        0.263852
fraud_Zulauf LLC                    0.389105
Length: 693, dtype: float64

In [42]:
# Setup final dataset

# combine all data
computed_cols = pd.read_csv("new_col_data.csv")
computed_cols.set_index("transNum", inplace=True)
df = transactions.join(computed_cols, on="transNum")
df = df.join(cardholders, on="creditCardNum")

# drop irrelevant columns
irrelevant_cols = [
    "transDate",
    "creditCardNum",
    "firstName",
    "lastName",
    "street",
    "zip",
    "latitude",
    "longitude",
    "dateOfBirth",
    "merchLatitude",
    "merchLongitude",
    "lastTransaction",
    "meanPurchasePrice",
    "numberOfTransactions",
    "sumSquareDiffs",
]
df.drop(columns=irrelevant_cols, inplace=True)

# encode gender
gender_mapping = {"M": 1, "F": 0}
df["gender_male"] = df["gender"].map(gender_mapping)
df.drop(columns=["gender"], inplace=True)

# encode business
grouped_transactions = df1.groupby("business")
total_transactions = grouped_transactions.size()
fraud_transactions = grouped_transactions["isFraud"].sum()
fraud_rate = (fraud_transactions / total_transactions) * 100
result_dict = fraud_rate.to_dict()

df["business_fraudrate"] = df["business"].map(result_dict)
df.drop(columns=["business"], inplace=True)

# encode category
grouped_transactions = df1.groupby("category")
total_transactions = grouped_transactions.size()
fraud_transactions = grouped_transactions["isFraud"].sum()
fraud_rate = (fraud_transactions / total_transactions) * 100
result_dict = fraud_rate.to_dict()

df["category_fraudrate"] = df["category"].map(result_dict)
df.drop(columns=["category"], inplace=True)

# encode city
grouped_transactions = df1.groupby("city")
total_transactions = grouped_transactions.size()
fraud_transactions = grouped_transactions["isFraud"].sum()
fraud_rate = (fraud_transactions / total_transactions) * 100
result_dict = fraud_rate.to_dict()

df["city_fraudrate"] = df["city"].map(result_dict)
df.drop(columns=["city"], inplace=True)

# encode state
grouped_transactions = df1.groupby("state")
total_transactions = grouped_transactions.size()
fraud_transactions = grouped_transactions["isFraud"].sum()
fraud_rate = (fraud_transactions / total_transactions) * 100
result_dict = fraud_rate.to_dict()

df["state_fraudrate"] = df["state"].map(result_dict)
df.drop(columns=["state"], inplace=True)

# encode job
grouped_transactions = df1.groupby("job")
total_transactions = grouped_transactions.size()
fraud_transactions = grouped_transactions["isFraud"].sum()
fraud_rate = (fraud_transactions / total_transactions) * 100
result_dict = fraud_rate.to_dict()

df["job_fraudrate"] = df["job"].map(result_dict)
df.drop(columns=["job"], inplace=True)

/var/folders/dy/nrpbxz756vv3_l8mp02k87k80000gn/T/ipykernel_47700/4106677937.py:35: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [43]:
df

,category,amount,isFraud,distanceFromHome,distanceFromLast,secondsSinceLast,priceSD,city,state,cityPop,job,historyOfFraud,gender_male,business_fraudrate
transNum,,,,,,,,,,,,,,
3fc1791ff1db94ce9540262d7eb973e4,misc_net,4.10,False,47.229861,-1.000000,-1,0.000000,Eureka,IL,6713,Archivist,False,0,2.247191
edbf65b8606b53fc48168c2daa5933e5,shopping_net,2.67,False,27.264981,-1.000000,-1,0.000000,Mallie,KY,798,Facilities manager,False,1,0.829876
2fcbde131cff833751dde53da54da6da,misc_pos,22.88,False,116.068684,-1.000000,-1,0.000000,Tomales,CA,337,Occupational psychologist,False,0,0.000000
bcae564f9a1a299dfbd1efd917499968,misc_pos,9.59,False,80.518979,-1.000000,-1,0.000000,Belmond,IA,3032,Community pharmacist,False,1,0.000000
5e42e35c255edb55a192b3f1f0600936,grocery_pos,129.06,False,65.543964,-1.000000,-1,0.000000,Shenandoah Junction,WV,1925,Systems developer,True,0,2.719033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16cfc30968a4ffc36f44a8fa4d98224f,kids_pets,72.97,False,40.897276,82.397940,13171,80.433603,Burke,VA,43102,Health visitor,False,1,0.000000
9c68a7b61b6d9a1afd3db343f6943e0e,travel,7.50,False,29.410519,103.695127,8198,64.068313,San Diego,CA,1241364,Chartered public finance accountant,False,1,0.806452
e715a05b1aab3b771d6c22a0ef0e6fcd,personal_care,46.18,False,89.146589,40.983171,3723,82.925814,Jelm,WY,100,Public librarian,True,0,0.387597


In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 1: Load and preprocess the data
data = df1.copy()
X = data.drop(columns=["isFraud"])
y = data["isFraud"]


# One-hot encoding
df_encoded = pd.get_dummies(df, columns=["A"])


# Step 2: Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Apply PCA
pca = PCA(n_components=20)  # Choose the number of components
X_pca = pca.fit_transform(X_scaled)

# Step 4: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42
)

# Step 5: Fit linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Step 6: Evaluate the model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
print(f"Training R-squared: {train_score:.3f}")
print(f"Test R-squared: {test_score:.3f}")